In [ ]:
%load_ext autoreload
%autoreload 2
 
%load_ext magic_duckdb

from magic_duckdb import logging_init

logging_init.log_to_file("magic_duckdb2.log")


In [ ]:
%dql select sometablename2.my asdasd

In [ ]:
import duckdb 
from pandas import DataFrame
import numpy as np
from magic_duckdb import magic
from magic_duckdb.autocompletion import DqlCustomCompleter
from types import SimpleNamespace
con = duckdb.connect()

some_tablename = "tablenamereallylong"
# create some tables
simpledf = DataFrame(np.random.randn(10,20))

con.sql(f"CREATE OR REPLACE TABLE {some_tablename} as select * from simpledf")
con.sql("create table sometablename2 as select * from range(10) t(my_column_1)")
con.sql("create table longtablenameishardtomakeup as select * from range(10) t(d)")
con.sql("show tables").df()

# test autocompletion
magic.connection = con
completer = DqlCustomCompleter(shell=None)

# completer finds the table names
event = SimpleNamespace(line='%dql s', symbol='s', command='%dql', text_until_cursor='%dql s')
results = completer.line_completer(event)
#display(results)
assert some_tablename in results
assert "SELECT"  in results

event = SimpleNamespace(line='%dql s', symbol='s', command='%dql', text_until_cursor='%dql select * from')
results = completer.line_completer(event)
assert some_tablename in results
assert "SELECT" not in results

# Tablename doesn't exist
event = SimpleNamespace(line='%dql s', symbol='s', command='%dql', text_until_cursor='%dql select blah.')
results = completer.line_completer(event)
assert len(results) == 0


event = SimpleNamespace(line='%dql tablenamereallylong.', symbol='tablenamereallylong.', command='%dql', text_until_cursor='%dql  tablenamereallylong.')
results = completer.line_completer(event)
assert "19" in results

event = SimpleNamespace(line='%dql select tablenamereallylong. from tablenamereallylong', symbol='tablenamereallylong.', command='%dql', text_until_cursor='%dql select sometablename2.')
results = completer.line_completer(event)
assert "my_column_1" in results

event = SimpleNamespace(line='%dql s', symbol='s', command='%dql', text_until_cursor=f'%dql select {some_tablename}.')
results = completer.line_completer(event)
assert "19" in results
# Tablename doesn't exist



In [ ]:
# Test using IPCompleter
# This uses the full IPCompleter stack, so is more realistic in terms of results

from IPython.core.getipython import get_ipython

ip = get_ipython()
event = SimpleNamespace(line='%dql get_ipython', symbol='g', command='%dql get_ipython', text_until_cursor=f'%dql get_ipython', cursor_pos=6)

r = ip.Completer.complete(line_buffer="%dql crea")
assert "CREATE TABLE" in r[1]

r = ip.Completer.complete(line_buffer="%dql sometablename2.")
display( r[1])



In [ ]:
%dql select * from some_tablename.

In [ ]:
import re

pragma_before = re.compile(r"(?si).*pragma\s*")

pragma_before.match(" pragma ") is not None

from magic_duckdb import magic as m

print(m.last_obj)



In [ ]:
import re 
lastword = re.compile(r"(?si)((.*\s)|(^))(\S+)\s+.*")

lastword.match("select asdasd").group(4)
